# TextBlob Sentiment Analysis

This notebook demonstrates using TextBlob to compute sentiment polarity and subjectivity for customer reviews. TextBlob provides a simple API to extract these scores from text.

## Step 1 - Import Dependencies

Import pandas for data handling and `TextBlob` (and `Word`) from the textblob library for sentiment analysis utilities.

In [ ]:
from textblob import TextBlob, Word
import pandas as pd

## Step 2 - Get Documents

Read the reviews Excel sheet into a pandas DataFrame. Calling `head()` shows the first few rows to verify the data loaded correctly.

In [ ]:
reviews_df = pd.read_excel('data/reviews.xlsx')
reviews_df.head()

## Step 3 - Define Processing Function

The `process_row` function computes two values for each review text:
- Polarity: how positive/negative the sentiment is, ranging from -1.0 (very negative) to +1.0 (very positive).
- Subjectivity: how opinionated the text is, ranging from 0.0 (very objective) to 1.0 (very subjective).

It then trims each float to two decimal places for readability and returns a tuple `(polarity, subjectivity)`.

In [ ]:
def process_row(row):
    polarity = TextBlob(row['Review Text']).sentiment.polarity
    subjectivity = TextBlob(row['Review Text']).sentiment.subjectivity

    factor = 10 ** 2
    polarity_final = int(polarity * factor) / factor
    subjectivity_final = int(subjectivity * factor) / factor
    return polarity_final, subjectivity_final

## Step 4 - Results

Apply the function to each row to produce a Series of `(polarity, subjectivity)` tuples, then print the results for quick inspection.

In [ ]:
results = reviews_df.apply(process_row, axis=1)
print(results)

### Notes

- TextBlob’s sentiment is rule/lexicon-based (PatternAnalyzer) and may miss context, sarcasm, or complex negations.
- Consider basic preprocessing (e.g., lowercasing, removing punctuation) if your data is noisy.
- Polarity and subjectivity are independent measures; a text can be highly subjective yet neutral in polarity.
- For more advanced handling (e.g., domain adaptation, context), consider model-based approaches.